<a href="https://colab.research.google.com/github/ALOK158/AI_Text_Detector/blob/main/GPT_FROM_SCRATCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
import pandas as pd
import numpy as np

In [16]:
#Lets start with our data set that is of tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-21 05:27:17--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.008s  

2025-06-21 05:27:17 (137 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [17]:
# Lets read the text
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [18]:
print("Length of the dataset in characters: ", len(text))


Length of the dataset in characters:  1115394


In [19]:
#First 1000 char
print(text[:10000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [20]:
# Get all uniuqe char in the text
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(len(chars))



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


##Tokenizing

Simplest way of tokenization- char level tokenization

In future we will explore more in this for to improve our model

In [21]:
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode=lambda s:[stoi[c] for c in s]
decode=lambda l:''.join([itos[i] for i in l])


In [22]:
# lets encode our Shakespeare Text
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape, data.type)

torch.Size([1115394]) <built-in method type of Tensor object at 0x7dbd6ac0d790>


Lets Split the data in Traing and Validation

In [23]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [24]:
block_size=8# sparese data in 8 times

In [25]:
x= train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
  context=x[:t]
  target=y[t]
  print(f"when input is {context} the target is {target}")

when input is tensor([], dtype=torch.int64) the target is 47
when input is tensor([18]) the target is 56
when input is tensor([18, 47]) the target is 57
when input is tensor([18, 47, 56]) the target is 58
when input is tensor([18, 47, 56, 57]) the target is 1
when input is tensor([18, 47, 56, 57, 58]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 58


splitting our data in smaller chunks

In [26]:
torch.manual_seed(1337)
batch_size=4# Independent sequnece gets proccess in pararell
block_size=8 # Max context length

def get_batch(split):
  data=train_data if split=='train' else val_data
  ix=torch.randint(len(data)- block_size, (batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+1+block_size] for i in ix])
  return x, y

xb, yb= get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----


In [29]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets is None:
            loss = None
        else:
          B, T, C=logits.shape
          logits=logits.view(B*T,C)
          targets=targets.view(B*T)
          loss=F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in current context
        for _ in range(max_new_tokens):
          #do the prediciton
          logits, loss=self(idx)
          #focus only on last time step
          logits=logits[:,-1,:]
          #apply softmax to do prediction
          probs=F.softmax(logits, dim=-1)# (B,C)
          #sample from distribution
          idx_next=torch.multinomial(probs, num_samples=1)#(B,1)
          #append sample index to running sequnece
          idx=torch.cat((idx, idx_next), dim=1)#(B,T+1)
        return idx



m= BigramLanguageModel(vocab_size)
logits, loss=m(xb,yb)

print(loss)

idx=torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx=torch.zeros((1,1), dtype =torch.long), max_new_tokens=100)[0].tolist()))

tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


Lets train the model

In [39]:
from torch.optim import AdamW
# Create pytorch optimizer
optimizer=torch.optim=AdamW(m.parameters(), lr=1e-4)


In [42]:
batch_size=16
for steps in range(10000):
  #sample a batch of data
  xb,yb=get_batch('train')
  #evaluate the loss
  logits, loss=m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.864046812057495


In [43]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype =torch.long), max_new_tokens=100)[0].tolist()))


W?ue tu, bl3Bw'mBTatowrtlaid lowatyte ind Ifrm se?ush FPy.
VgigHJattlveer pg!ze W!
'O,SPHd p$k,
DKuM
